In [ ]:
import tensorflow as tf
import pathlib
from tensorflow import keras, lite
from tensorflow.keras.layers import Dense, Flatten
print(tf.__version__)


In [ ]:
IMG_SIZE = 96
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 32
CLASSES = 4
EPOCHS = 10
LEARNING_RATE = 0.0034


In [ ]:
data_dir = pathlib.Path('Dataset/Training')
test_dir = pathlib.Path('Dataset/Testing')
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                                data_dir,
                                                                validation_split = 0.2,
                                                                subset = 'training',
                                                                seed = 123,
                                                                image_size = (IMG_SIZE,IMG_SIZE),
                                                                batch_size = BATCH_SIZE
                                                              )
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                            data_dir,
                                                            validation_split = 0.2,
                                                            subset = 'validation',
                                                            seed = 123,
                                                            image_size = (IMG_SIZE,IMG_SIZE),
                                                            batch_size = BATCH_SIZE
                                                            )
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                            test_dir,
                                                            seed = 123,
                                                            image_size = (IMG_SIZE,IMG_SIZE),
                                                            batch_size = BATCH_SIZE
                                                            )

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 3))
for images, labels in train_ds.take(1):
  for i in range(32):
    ax = plt.subplot(2, 16, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 7))
for images, labels in test_ds.take(1):
  for i in range(32):
    ax = plt.subplot(4, 8, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(test_ds.class_names[labels[i]])
    plt.axis("off")

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


base_model = MobileNetV2(weights='imagenet',
                        input_shape=(IMG_SIZE, IMG_SIZE, 3),
                        # alpha=0.35,
                        include_top=False)

base_model.trainable = False

def model_maker():
  inputs = keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))
  x = preprocess_input(inputs)
  x = base_model(x, training=False)
  x = base_model.output
  flatten = Flatten()(x)
  outputs = Dense(CLASSES, activation='softmax')(flatten)
  model = keras.Model(base_model.input, outputs)
  return model

In [ ]:
model = model_maker()
model.summary()

In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = EPOCHS,
                    )

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# import numpy as np
# import cv2

# # multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

# #https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
# face_cascade = cv2.CascadeClassifier('Haar/haarcascade_frontalface_default.xml')
# #https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
# eye_cascade = cv2.CascadeClassifier('Haar/haarcascade_eye.xml')

# # mouth_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

# cap = cv2.VideoCapture(0)
# while 1:
#     ret, img = cap.read()
#     img = cv2.flip(img, 1)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.1, 3)

#     for (x,y,w,h) in faces:
#         roi_gray = gray[y:y+h, x:x+w]
#         roi_color = img[y:y+h, x:x+w]
#         roi_color = cv2.resize(roi_color, (96, 96))
#         eyes = eye_cascade.detectMultiScale(roi_gray)
#         for (ex,ey,ew,eh) in eyes:
#             cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
#             img_array = tf.keras.utils.img_to_array(roi_color)
#             img_array = tf.expand_dims(img_array, 0) # Create a batch

#             predictions = model.predict(img_array)
#             score = tf.nn.softmax(predictions[0])
#             text = "{} : {:.2f}".format(classes[np.argmax(score)], 100 * np.max(score))
#             img = cv2.putText(img, text, (x,y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)


#     cv2.imshow('img',img)
#     k = cv2.waitKey(1) & 0xff
#     if k == 27:
#         break


# cv2.destroyAllWindows()
# cv2.waitKey(1)
# cap.release()

In [ ]:
import os
class_names = test_ds.class_names
counter = [0] * len(class_names)
testing_path = "Dataset5/Testing/"
label = "Wardah"


label_path = os.path.join(testing_path, label)

for image in os.listdir(label_path):
    image_path = os.path.join(label_path, image)
    try:
        img = tf.keras.utils.load_img(
            image_path, target_size=(96, 96)
        )
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        print(
            "{} : {:.2f} \t\t\t {}"
            .format(class_names[np.argmax(score)], 100 * np.max(score), image_path)
        )
        counter[np.argmax(score)] += 1
    except:
        print("Error loading image :", image_path)
print(class_names)
print(counter)

In [ ]:
import numpy as np
class_names = test_ds.class_names
img = tf.keras.utils.load_img(
    "kanna.jpeg", target_size=(96, 96)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
import numpy as np

def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((images)).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.float32)]

# def representative_dataset():
#     for _ in range(100):
#       data = np.random.rand(1, 96, 96, 3)
#       yield [data.astype(np.float32)]

# Convert the tflite.
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
with open('trained_01.tflite', 'wb') as f:
  f.write(tflite_quant_model)